---
title: "Latent Variable Models 🚧"
description: "Models that relate a set of observable variables to a set of unobserved (latent) variables."
categories: [Structural Equation Modeling, Factor Analysis]
image: "Figures/25.png"
order: 21
---

## General Principles

In some scenarios, the observed data does not directly reflect the underlying structure or factors influencing the outcome. Instead, _latent variables_—variables that are not directly observed but are inferred from the data—can help model this hidden structure. These latent variables capture unobserved factors that affect the relationship between predictors (_X_) and the outcome (_Y_).

We model the relationship between the predictor variables (_X_) and the outcome variable (_Y_) with a latent variable (_Z_) as follows:

$$
Y = f(X, Z) + \epsilon
$$

Where:
- _Y_ is the observed outcome variable.
- _X_ is the observed predictor variable(s).
- _Z_ is the latent (unobserved) variable, which we aim to infer.
- _f(X, Z)_ is the function that relates _X_ and _Z_ to _Y_.
- _\epsilon_ is the error term, typically assumed to be normally distributed with mean 0 and variance _\sigma^2_.

The latent variable _Z_ can represent various phenomena, such as group-level effects, time-varying trends, or individual-level factors, that are not captured by the observed predictors alone.

## Considerations

In Bayesian regression with latent variables, we consider the uncertainty in both the observed and latent variables. We declare prior distributions for the latent variables, in addition to the usual priors for regression coefficients and intercepts. These latent variables are often modeled using Gaussian distributions (_Normal_ priors) or more flexible distributions such as _Multivariate Normal_ for correlations among the latent variables.

The goal is to infer the posterior distribution over both the parameters and the latent variables, given the observed data.

## Example

Below is an example code snippet demonstrating Bayesian regression with latent variables using TensorFlow Probability:

::: {.panel-tabset group="language"}
## Python

In [ ]:
from BI import bi, jnp


# Setup device------------------------------------------------
m = bi(platform='cpu')

# Data Simulation ------------------------------------------------
NY = 4  # Number of dependent variables or outcomes (e.g., dimensions for latent variables)
NV = 8  # Number of observations or individual-level data points (e.g., subjects)
N = 100
K = 5
a = 0.5
# Generate the means and offsets for the data
# means: Generate random normal means for each of the NY outcomes
# offsets: Generate random normal offsets for each of the NV observations
means = m.dist.normal(0, 1, shape=(NY,), sample=True, seed=10)
offsets = m.dist.normal(0, 1, shape=(NV, 1), sample=True, seed=20)

Y2 = offsets + means

# Simulate individual-level random effects (e.g., random slopes or intercepts)
# b_individual: A matrix of size (N, K) where N is the number of individuals and K is the number of covariates
b_individual = m.dist.normal(0, 1, shape=(N, K), sample=True, seed=0)

# mu: Add an additional effect 'a' to the individual-level random effects 'b_individual'
# 'a' could represent a population-level effect or a baseline
mu = b_individual + a

# Convert Y2 to a JAX array for further computation in a JAX-based framework
Y2 = jnp.array(Y2)


# Set data ------------------------------------------------
dat = dict(
    NY = NY,
    NV = NV,
    Y2 = Y2
)
m.data_on_model = dat

# Define model ------------------------------------------------
def model(NY, NV, Y2):
    means = m.dist.normal(0, 1, shape=(NY,), name='means')
    offset = m.dist.normal(0, 1, shape=(NV, 1), name='offset')
    sigma = m.dist.exponential(1, shape=(NY,), name='sigma')
    tmp = jnp.tile(means, (NV, 1)).reshape(NV, NY)
    mu_l = tmp + offset
    m.dist.normal(mu_l, jnp.tile(sigma, [NV, 1]), obs=Y2)

# Run sampler ------------------------------------------------
m.fit(model)

# Summary ------------------------------------------------
m.summary()

## R
``` R
```
![](travaux-routiers.png){fig-align="center"}
:::

## Mathematical Details

![](travaux-routiers.png){fig-align="center"}
<!---
We can express the Bayesian latent variable model using probability distributions as follows:

$$
\begin{aligned}
& p(Y | X, Z, W, \sigma) = \text{Normal}(X \cdot W + Z, \sigma^2) \\
& p(Z) = \text{Normal}(0, \tau^2) \\
& p(W) = \text{Normal}(0, \alpha^2) \\
\end{aligned}
$$

Where:
- _p(Y | X, Z, W, \sigma)_ is the likelihood function for the observed outcome variable, which depends on both the observed predictor _X_ and the latent variable _Z_.
- _p(Z)_ is the prior distribution for the latent variable _Z_, often modeled as _Normal_ with a mean of 0 and variance _\tau^2_.
- _p(W)_ is the prior distribution for the regression coefficient(s) _W_, typically assumed to follow a _Normal_ distribution with mean 0 and variance _\alpha^2_.

The latent variable _Z_ introduces additional flexibility to the model, capturing unobserved influences on the outcome _Y_.

## Interpretation of Latent Variables

- **Latent Variable (_Z_)**: Represents hidden factors not captured by the observed variables, allowing the model to explain more of the variance in the outcome. For instance, in a psychological model, _Z_ might represent a latent trait such as intelligence or anxiety that influences the outcome.
  
- **Posterior Inference**: The posterior distribution of the latent variable _Z_ can give insights into how much the unobserved factors contribute to the outcome.

## Use Cases
- **Latent Factors in Psychometrics**: In psychometric models, latent variables represent traits or abilities that are not directly observed, such as cognitive ability or personality traits.
- **Time-Varying Effects**: Latent variables can represent unobserved time trends or individual-specific effects in time-series or longitudinal models.
- **Mixed Models**: In hierarchical or mixed models, latent variables can represent group-specific intercepts or slopes.

-->